<a href="https://colab.research.google.com/github/anjalipaladi/Named-Entity-Recognition/blob/main/Covid_Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("covid_fake.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['label'].value_counts()

In [ ]:
df.loc[5:15]

In [ ]:
df.isna().sum()

In [ ]:
df.loc[df['label']=="Fake",['label']]='FAKE'
df.loc[df['label']=="Fake",['label']]='FAKE'
df.loc[df['source']=='facebook',['source']]='Facebook'
df.text.fillna(df.title,inplace=True)
df.loc[5]['label']='FAKE'
df.loc[15]['label']='TRUE'
df.loc[43]['label']='FAKE'
df.loc[131]['label']='TRUE'
df.loc[242]['label']='FAKE'
df.title.fillna('missing',inplace=True)
df.source.fillna('missing',inplace=True)
df['title_text']=df['title']+""+df["text"]

In [ ]:
df.isna().sum()

In [ ]:
df['label'].value_counts()

In [ ]:
df.head()

In [ ]:
df['title_text'][3]

In [ ]:
def preprocessor(text):
  text=re.sub('<[^>]*>','',text)
  text=re.sub(r'[^\w\s]','',text)
  text=re.sub(r'[\n]','',text)
  text=text.lower()
  return text
df['title_text']=df['title_text'].apply(preprocessor)
df['title_text'][3]

In [ ]:
porter=PorterStemmer()
def tokenize_porter(text):
  return [porter.stem(word) for word in text.split()]

TF-IDF

In [ ]:
tfidf=TfidfVectorizer(strip_accents=None,
                      lowercase=False,
                      preprocessor=None,
                      tokenizer=tokenize_porter,
                      use_idf=True,
                      norm='l2',
                      smooth_idf=True)
X=tfidf.fit_transform(df['title_text'])
y=df.label.values

In [ ]:
X.shape

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.3,shuffle=False)


In [ ]:
clf=LogisticRegressionCV(cv=5,scoring='accuracy',random_state=0,n_jobs=-1,verbose=0,max_iter=300)
clf.fit(X_train,y_train)
fake_news_model=open('fake_news_model.sav','wb')
pickle.dump(clf,fake_news_model)
fake_news_model.close()

Model Evaluation

In [ ]:
filename='fake_news_model.sav'
saved_clf=pickle.load(open(filename,'rb'))
saved_clf.score(X_test,y_test)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
y_pred=clf.predict(X_test)
print("---Test Set Results")
print(classification_report(y_test,y_pred))
